# Probabilistic Methods (Discrete)

What is it? 
- Focusing on how to identify anomalies in our discrete variables

Why do we care?
- Allows us to find items, events, or observations which are significantly different from the remaining data and determine whether or not they belong

How do we do it? 

- **count**: the number of times each unique value appears in the dataset
- **frequencies**: the number of times each unique value appears in the dataset as a percentage of the total
- **conditional probability**: the likelihood of an event or outcome occurring, based on the occurrence of a previous event or outcome


Examples:

- count: How many times we see an ip address in the dataset
- frequncy: What is probability of ip address showing up in the dataset
- conditional probability: Given an ip address, what is prob of a particular status (e.g authentication failure)

### Scenario

Use codeup's access logs to determine if anyone is accessing the curriculum when they shouldn't be. This data is from codeup's first data science cohort, Ada. Each row is one request to the server.

In [22]:
#do the data things!
import numpy as np
import pandas as pd

#visualize
import matplotlib.pyplot as plt
import seaborn as sns

#my sql creds
import env

## Acquire

<div class="alert alert-block alert-info">
<b>NOTE: </b> 
    
`pd.read_sql` operates differently in the newest update
    
New process: 
1. import `text` and `create_engine` from `sqlalchemy`
2. create url (same way as before)
3. create engine with url and connect
4. specify query as text
5. send query and engine into `pd.read_sql`
    
</div>

1. import text and create_engine from sqlalchemy

In [35]:
from sqlalchemy import text, create_engine

2. create url (same way as before)


In [36]:
url = env.get_db_url('logs')

3. create engine with url and connect

In [37]:
conn = create_engine(url).connect()

4. specify query as text

In [38]:
query = text('show tables')

5. send query and connection into pd.read_sql

In [39]:
pd.read_sql(query, conn)

,Tables_in_logs
0,api_access


In [40]:
pd.read_sql("show tables", conn)

,Tables_in_logs
0,api_access


In [45]:
df = pd.read_sql("select * from api_access", conn)

In [46]:
df.head()

,entry
0,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
1,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
2,97.105.19.58 - - [16/Apr/2019:19:34:44 +0000] ...
3,97.105.19.58 - - [16/Apr/2019:19:34:46 +0000] ...
4,97.105.19.58 - - [16/Apr/2019:19:34:48 +0000] ...


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13974 entries, 0 to 13973
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   entry   13974 non-null  object
dtypes: object(1)
memory usage: 109.3+ KB


### Prepare

the data needs to be parsed out so we can see each piece of the access log

In [48]:
#accessing one row
df.entry[0]

'97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] "GET /api/v1/sales?page=81 HTTP/1.1" 200 512495 "-" "python-requests/2.21.0"'

In [49]:
#separating the elements in one row
df.entry[0].split()

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [50]:
#save it
parts = df.entry[0].split()
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [51]:
df.entry.str.split()

0        [97.105.19.58, -, -, [16/Apr/2019:19:34:42, +0...
1        [97.105.19.58, -, -, [16/Apr/2019:19:34:42, +0...
2        [97.105.19.58, -, -, [16/Apr/2019:19:34:44, +0...
3        [97.105.19.58, -, -, [16/Apr/2019:19:34:46, +0...
4        [97.105.19.58, -, -, [16/Apr/2019:19:34:48, +0...
                               ...                        
13969    [97.105.19.58, -, -, [17/Apr/2019:12:55:12, +0...
13970    [97.105.19.58, -, -, [17/Apr/2019:12:55:12, +0...
13971    [97.105.19.58, -, -, [17/Apr/2019:12:55:13, +0...
13972    [97.105.19.58, -, -, [17/Apr/2019:12:55:13, +0...
13973    [97.105.19.58, -, -, [17/Apr/2019:12:55:14, +0...
Name: entry, Length: 13974, dtype: object

In [52]:
#create dictionary to hold results
output = {}

In [53]:
#pull out ip address
output['ip'] = parts[0]

In [54]:
output

{'ip': '97.105.19.58'}

In [55]:
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [56]:
df.entry.str.split(expand=True)[1].unique()

array(['-'], dtype=object)

In [57]:
#pull out time # remove [] by using [1:]
output['time'] = parts[3][1:].replace(":", "")

In [58]:
output

{'ip': '97.105.19.58', 'time': '16/Apr/2019193442'}

In [59]:
#pull out request method
df.entry.str.split(expand=True)[4].unique()

array(['+0000]'], dtype=object)

In [60]:
df.entry.str.split(expand=True)[5].unique()

array(['"GET'], dtype=object)

In [61]:
output['request_method'] = parts[5].strip('"')

In [62]:
#pull out request path
output['request_path'] = parts[6]

In [63]:
output

{'ip': '97.105.19.58',
 'time': '16/Apr/2019193442',
 'request_method': 'GET',
 'request_path': '/api/v1/sales?page=81'}

In [64]:
#pull our http version
output['http_version'] = parts[7][:-1]

In [11]:
#pull out status code
output['status_code'] = parts[8]

In [12]:
# pull out size
output['size'] = parts[9]

In [ ]:
for col in df.entry.str.split(expand=True).iloc[:,11:].columns:
    print(df.entry.str.split(expand=True).unique())

In [13]:
#combine request agents together
output['request_agent'] = parts[11:]

In [34]:
' '.join(parts[11:]).strip('"')

NameError: name 'parts' is not defined

In [ ]:
parts

In [65]:
# function to deal with parsing one entry in our log data
# works on one string
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).strip('"')
    return pd.Series(output)

In [66]:
#apply string function to every element in a series
df.entry.apply(parse_log_entry)

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0
...,...,...,...,...,...,...,...,...
13969,97.105.19.58,17/Apr/2019 12:55:12,GET,/api/v1/sales?page=62,HTTP/1.1,200,511544,python-requests/2.21.0
13970,97.105.19.58,17/Apr/2019 12:55:12,GET,/api/v1/sales?page=65,HTTP/1.1,200,511201,python-requests/2.21.0
13971,97.105.19.58,17/Apr/2019 12:55:13,GET,/api/v1/sales?page=73,HTTP/1.1,200,511781,python-requests/2.21.0
13972,97.105.19.58,17/Apr/2019 12:55:13,GET,/api/v1/sales?page=69,HTTP/1.1,200,510136,python-requests/2.21.0


### For the sake of lesson, adding some anomalies

In [16]:
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", 
     '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", 
     '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", 
     '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", 
     '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

NameError: name 'df' is not defined

In [ ]:
df = pd.concat([df,new])

In [ ]:
df

#### making new size column based mb

In [ ]:
#divide by 1024 twice


## Detecting Anomalies in Discrete Variables

- count
- frequency
- visualize
- conditional probability

###  count

let's see how many times each ip address comes up in our dataset

In [ ]:
#save it into a dataframe


### frequency

let's now calculate the probabity for each ip 

ip_prob = count for each ip / total count in the dataframe

In [ ]:
#divide


In [ ]:
#best function


In [ ]:
#save it to a df


#### merge em

In [ ]:
+

### visualize them

In [ ]:


# plt.title('counts of each ip address')
# plt.show()

In [ ]:

# plt.title('percent of each ip address')
# plt.show()

### conditional probability

what is the probability of event A happening given that event B has happened? 

$P(A|B) = {P(A \cap B) \over P(B)}$

aka: the probability of both event A and event B happening divided by the probability of only B happening

#### our access logs

What is probability of a certain status code given an IP address?

HTTP Status Codes

    200: ok
    3xx: redirects
    4xx: client level errors -- the requester did something wrong
    5xx: server level errors -- the server did something wrong


In [ ]:
#get the counts for ip and status_code


In [ ]:
#probability of ip and status code


In [ ]:
#probability of ip


In [ ]:
#probability of ip given a status code


In [ ]:
# Given a particular ip, what is probability of a certain status code


#### Cases where the probability is < 100% -- this means that multiple different status codes were encountered from the given ip address

#### Status codes other than 200. 200 is the most common response code in our dataset by far.